**step 1)**

In [1]:
!pip install librosa
!pip install torch torchvision torchaudio   # required for transformers + audio
!pip install transformers
!pip install accelerate
!pip install sentencepiece


**step 2)**

In [2]:
from io import BytesIO
from urllib.request import urlopen
import librosa
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor




**step 3)**

In [3]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")



preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

**step 4) ** **Example No 1**

In [42]:
conversation = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/glass-breaking-151256.mp3"},
        {"type": "text", "text": "What's that sound?"},
    ]},
    {"role": "assistant", "content": "It is the sound of glass shattering."},
    {"role": "user", "content": [
        {"type": "text", "text": "What can you do when you hear that?"},
    ]},
    {"role": "assistant", "content": "Stay alert and cautious, and check if anyone is hurt or if there is any damage to property."},
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/1272-128104-0000.flac"},
        {"type": "text", "text": "What does the person say?"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)


**step 4) ** **Example No 2**

In [160]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "http://raw.githack.com/dhananjay431/db_ai/main/How_Large_Language_Models_Work.mp3"},
        {"type": "text", "text": "what is ollama model?"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)

**step 4) ** **Example No 3**

In [194]:

conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "http://raw.githack.com/dhananjay431/db_ai/main/AI.mp3"},
        {"type": "text", "text": "create Summary for type of AI?"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)

**step 5)**

In [195]:
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                # Check if it's a local file path or a URL
                if ele['audio_url'].startswith('./') or ele['audio_url'].startswith('/'):
                    # Load local file
                    audios.append(
                        librosa.load(
                            ele['audio_url'],
                            sr=processor.feature_extractor.sampling_rate)[0]
                    )
                else:
                    # Load from URL
                    audios.append(
                        librosa.load(
                            BytesIO(urlopen(ele['audio_url']).read()),
                            sr=processor.feature_extractor.sampling_rate)[0]
                    )

**step 6)**

In [196]:
inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs.input_ids = inputs.input_ids.to("cuda")



**step 7)**

In [197]:
inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs = {k: v.to("cuda") for k, v in inputs.items()}

generate_ids = model.generate(**inputs, max_length=256)
generate_ids = generate_ids[:, inputs['input_ids'].size(1):]

**step 8)**

In [198]:
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


In [199]:
response

"There are many different types of artificial intelligence (AI), each with its own unique characteristics and capabilities. Some examples include:\n\n  * Narrow or weak AI, which is designed to perform a specific task or set of tasks. Examples include Siri, Alexa, and chatbots.\n  * General or strong AI, which has the ability to perform any intellectual task that a human can do. This type of AI does not yet exist, but it is the goal of many researchers in the field.\n  * Superintelligent AI, which is an AI system that is significantly more intelligent than the average human. This type of AI could potentially pose risks to humanity if it were to be created, as it could outsmart humans and potentially make decisions that are harmful to them.\n  * Artificial general intelligence (AGI), which is a type of AI that is capable of performing any intellectual task that a human can do. It is considered the Holy Grail of AI research, as it would be able to solve problems that are currently beyond